In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import numpy as np
import shapely.geometry as shpg
from shapely.ops import linemerge
from shapely import set_precision
import os
import sys
import csv
import json
import logging
import shutil
import utm

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
sys.path.append(os.getcwd() + '/../..')
from utils import mkdir, open_zip_shapefile, open_tar_shapefile, haversine, correct_geoms, fix_overaps

In [ ]:
log = logging.getLogger('papermill')
logging.basicConfig(level='INFO', format="%(message)s")

## Files and storage paths

In [ ]:
# Region of interest
reg = 1

# go down from rgi7_scripts/workflow
data_dir = '../../../../rgi7_data/'

# Input dirctory
input_dir = os.path.join(data_dir, 'l3_rgi7a_tar')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l4_rgi7b0'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l4_rgi7b0_tar'))

In [ ]:
# Parameters
reg = 7


In [ ]:
reg_str = f'{reg:02d}'

In [ ]:
# RGI v6 file for comparison later 
rgi6_files = {
    '01': '01_rgi60_Alaska.zip',
    '02': '02_rgi60_WesternCanadaUS.zip',
    '03': '03_rgi60_ArcticCanadaNorth.zip',
    '04': '04_rgi60_ArcticCanadaNorth.zip',
    '05': '05_rgi60_GreenlandPeriphery.zip',
    '06': '06_rgi60_Iceland.zip',
    '07': '07_rgi60_Svalbard.zip',
    '08': '07_rgi60_Scandinavia.zip',
    '09': '09_rgi60_RussianArctic.zip',
    '10': '10_rgi60_NorthAsia.zip',
    '11': '11_rgi60_CentralEurope.zip',
    '12': '12_rgi60_CaucasusMiddleEast.zip',
    '13': '13_rgi60_CentralAsia.zip',
    '14': '14_rgi60_SouthAsiaWest.zip',
    '15': '15_rgi60_SouthAsiaEast.zip',
    '16': '16_rgi60_LowLatitudes.zip',
    '17': '17_rgi60_SouthernAndes.zip',
    '18': '18_rgi60_NewZealand.zip',
    '19': '19_rgi60_AntarcticSubantarctic.zip',
}
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', rgi6_files[reg_str])

### Load the input data

In [ ]:
# Read files
shp = open_tar_shapefile(input_dir + f'/RGI{reg:02d}.tar.gz')

In [ ]:
orig_attrs = pd.DataFrame(shp.drop('geometry', axis=1))
orig_attrs.T;

In [ ]:
if 'conn_lvl' not in shp:
    print('Add conn_lvl')
    shp['conn_lvl'] = 0

In [ ]:
odf = shp.copy()

In [ ]:
rename = {'area':'area_km2', 'CenLon': 'cenlon', 'CenLat': 'cenlat', 'glac_id':'glims_id'}
odf = odf.rename(rename, axis=1)

Recompute area and center point to be sure:

In [ ]:
def xy_coord(geom):
    x, y = geom.xy
    return x[0], y[0]

rp = odf.representative_point()

coordinates = np.array(list(rp.apply(xy_coord)))
odf['cenlon'] = coordinates[:, 0]
odf['cenlat'] = coordinates[:, 1]

odf['area_km2'] = odf.to_crs({'proj':'cea'}).area * 1e-6

In [ ]:
odf['glac_name'] = odf['glac_name'].where(odf['glac_name'] != 'None', '')

## Metadata 

In [ ]:
with open('../rgi7_attributes_metadata.json', 'r') as infile:
    meta_glacier_product = json.load(infile)

In [ ]:
odf_new = odf[[]].copy().reset_index(drop=True)

for col, attrs in meta_glacier_product.items():
    if col not in odf:
        if attrs['datatype'] == 'str':
            odf_new[col] = ''
        elif attrs['datatype'] == 'float':
            odf_new[col] = np.NaN
        elif attrs['datatype'] == 'int':
            if col in ['term_type', 'surge_type', 'aspect_sec']:
                odf_new[col] = 9 
            else:
                odf_new[col] = -999 
    else: 
        if attrs['datatype']:
            odf_new[col] = odf[col].astype(attrs['datatype'])
        else: 
            odf_new[col] = odf[col]

In [ ]:
odf_new = gpd.GeoDataFrame(odf_new)
odf_new.crs = odf.crs
odf_new = odf_new.reset_index(drop=True)
odf_new.iloc[:1].T;

## Regions, subregions and RGI IDs

In [ ]:
odf_new['o1region'] = f'{reg:02d}'

In [ ]:
sreg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O2Regions')
sreg = gpd.read_file(sreg_file)
sreg = sreg.loc[sreg.o1region == f'{reg:02d}']
sreg;

In [ ]:
baseid = f'RGI2000-v7.0-G-{reg:02d}-'
count = 1

rp = odf_new.representative_point()
rp = rp.to_frame('geometry')
rp['orig_index'] = odf_new.index

total = 0

for i, s in sreg.iterrows():
    
    intersect = gpd.overlay(rp, sreg.loc[[i]], how='intersection')
    odf_sreg = odf_new.loc[intersect['orig_index']].copy()
    odf_sreg['o2region'] = s['o2region']
    
    total += len(odf_sreg)
    
    if len(odf_sreg) == 0:
        # 19-05 Ross Ice Shelf one of them
        continue
    
    # Ids generation
    # Left most point and we start from there
    slon, slat = odf_sreg.loc[odf_sreg.cenlon == odf_sreg.cenlon.min()][['cenlon', 'cenlat']].iloc[0].values
    
    todo = odf_sreg.index.values
    todo_lon = odf_sreg['cenlon'].values
    todo_lat = odf_sreg['cenlat'].values
    ids = []
    while len(todo) > 0:
        dis = haversine(slon, slat, todo_lon, todo_lat)
        idm = np.argmin(dis)
        ids.append(todo[idm])
        slon, slat = todo_lon[idm], todo_lat[idm]
        todo = np.delete(todo, idm)
        todo_lon = np.delete(todo_lon, idm)
        todo_lat = np.delete(todo_lat, idm)
    
    assert len(ids) == len(odf_sreg)
    odf_sreg = odf_sreg.loc[ids].copy()
    
    odf_sreg['rgi_id'] = [baseid + f'{l+count:05d}' for l in range(len(odf_sreg))]
    count += len(odf_sreg)
    odf_new.loc[odf_sreg.index, 'rgi_id'] = odf_sreg['rgi_id']
    odf_new.loc[odf_sreg.index, 'o2region'] = odf_sreg['o2region']
    
odf_new = odf_new.sort_values(by='rgi_id').reset_index(drop=True)

assert odf_new['o2region'].isnull().sum() == 0
assert int(odf_new.iloc[-1]['rgi_id'].split('-')[-1]) == odf_new.iloc[-1].name + 1 
if len(odf_new['o2region'].unique()) != len(sreg):
    log.warning(f'RGI{reg:02d}: some subregions have NO glaciers in them')
else:
    log.info(f'RGI{reg:02d}: all subregions have glaciers in them')
assert len(odf_new) == len(odf)

In [ ]:
# UTM zone
utms = []
for lat, lon in zip(odf_new.cenlat, odf_new.cenlon):
    _, _, nz, _ = utm.from_latlon(lat, lon)
    utms.append(nz)
odf_new['utm_zone'] = utms

In [ ]:
nopoly = odf_new.loc[odf_new.type != 'Polygon']
assert len(nopoly) == 0

In [ ]:
odf_new.iloc[:1].T;

## Links to RGI6

In [ ]:
import overlaps_helpers

In [ ]:
# load RGI6 data
from utils import open_zip_shapefile
rgi6 = open_zip_shapefile(rgi6_reg_file)

In [ ]:
# Compute RGI7 - RGI6 overlaps
overlaps = overlaps_helpers.compute_cross_overlaps(odf_new.geometry, rgi6.geometry)

In [ ]:
# Add more stats
overlaps['area'] = overlaps['geometry'].to_crs({'proj':'cea'}).area * 1e-6
overlaps['i'] = odf_new['rgi_id'].iloc[overlaps['i']].values
overlaps['j'] = rgi6['RGIId'].iloc[overlaps['j']].values

In [ ]:
# Filter by minimum area. See https://github.com/ezwelty/rgi_links/issues/6
overlaps = overlaps[overlaps['area'] > 200].copy()

In [ ]:
# Count number of direct relatives (i.e. 1:1, n:1, 1:n, n:n)
overlaps['in'], overlaps['jn'] = overlaps_helpers.count_pair_relations(
  overlaps['i'], overlaps['j']
)
# Label clusters of (directly and indirectly-related) pairs
overlaps['cluster'] = overlaps_helpers.label_pair_clusters(overlaps['i'], overlaps['j'])

In [ ]:
# Remove geometry for now
odf_links = overlaps[['i', 'j', 'area', 'i_area_fraction', 'j_area_fraction', 'cluster', 'in', 'jn']].copy()
odf_links.columns = ['rgi7_id', 'rgi6_id', 'overlap_area_km2', 'rgi7_area_fraction', 'rgi6_area_fraction', 'cluster_id', 'n_rgi7', 'n_rgi6']

In [ ]:
odf_links;

## Submission metadata

In [ ]:
with open('../rgi7_submission_info_metadata.json', 'r') as infile:
    meta_sub = json.load(infile)

In [ ]:
subm_id = orig_attrs['subm_id'].unique()
odf_subm = pd.DataFrame()
for sid in subm_id:
    sel = orig_attrs.loc[orig_attrs['subm_id'] == sid]
    for k in meta_sub.keys():
        if k == 'subm_id':
            continue
        attrs = meta_sub[k]
        if k not in sel:
            if attrs['datatype'] == 'str':
                odf_subm.loc[int(sid), k] = ''
            elif attrs['datatype'] == 'float':
                odf_subm.loc[int(sid), k] = np.NaN
            elif attrs['datatype'] == 'int':
                odf_subm.loc[int(sid), k] = -999
        else: 
            assert len(sel[k].unique()==1), f'{k} has non unique values'
            if attrs['datatype']:
                data = sel[k].astype(attrs['datatype']).iloc[0]
            else: 
                data = sel[k].iloc[0]
                
            if attrs['datatype'] == 'str':
                # Clean
                data = data.strip().lstrip(';').strip()
                
            odf_subm.loc[int(sid), k] = data
            
    odf_subm.loc[int(sid), 'n_outlines'] = len(sel)
    odf_subm.loc[int(sid), 'area_km2'] = sel['area'].sum() * 1e-6
        
odf_subm.index.name = 'subm_id' 
odf_subm['n_outlines'] = odf_subm['n_outlines'].astype(int)
odf_subm['rc_id'] = odf_subm['rc_id'].astype(int)
odf_subm = odf_subm.sort_index()
odf_subm;

## Intersects product 

In [ ]:
import warnings
warnings.filterwarnings('default')

from shapely.errors import GEOSException

In [ ]:
# define how the output should look like
odf_intersects_cols = ['rgi_id_1', 'rgi_id_2', 'geometry']
odf_intersects = gpd.GeoDataFrame(columns=odf_intersects_cols)
odf_intersects.crs = odf_new.crs

# this precision is needed to avoid unwanted side
# effects due to floating point representation of
# polygon coordinates
precision = 1e-9

# this creates r-tree spatial indices for a fast search for potential intersects
# e.g. see https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
spatial_index = odf_new.sindex

for counter, major in odf_new.iterrows():
    
    if counter % 10 == 0 or counter == len(odf_new)-1:
        print(f"[{len(odf_new)}] {counter}", end="\r", flush=True)

    # find possible intersects using spatial indexing
    possible_intersects_index = list(spatial_index.query(major.geometry))
    possible_intersects = odf_new.iloc[possible_intersects_index]

    # exclude the major geometry itself
    possible_intersects = possible_intersects.loc[possible_intersects.rgi_id != major.rgi_id]

    # run true intersection query only on possible intersects
    try:
        actual_intersects = possible_intersects[possible_intersects.intersects(major.geometry)]
    except GEOSException:
        to_loc = []
        for ki, potential_inter in possible_intersects.iterrows():
            if set_precision(potential_inter.geometry, precision).intersects(set_precision(major.geometry, precision)):
                to_loc.append(ki)
        actual_intersects = possible_intersects.loc[to_loc]
    for _, neighbor in actual_intersects.iterrows():
        # Already computed?
        if neighbor.rgi_id in odf_intersects.rgi_id_1.values:
            continue

        # Here set new precision of geometries before intersecting,
        # this avoids side effects due to floating point
        # representation of coordinates (e.g. result is a polygon
        # instead of a line)
        mult_intersect = set_precision(major.geometry, precision).intersection(
            set_precision(neighbor.geometry, precision))

        # checks that floating point representation is ok
        if isinstance(mult_intersect, shpg.Polygon):
            # Check area and remove - should be fairly rare
            tmp = gpd.GeoDataFrame(geometry=[mult_intersect], crs=odf_new.crs)
            area = tmp.to_crs({'proj':'cea'}).area[0]
            assert area < 10
            continue

        if isinstance(mult_intersect, shpg.Point):
            continue
        if isinstance(mult_intersect, shpg.linestring.LineString):
            mult_intersect = shpg.MultiLineString([mult_intersect])
        if len(mult_intersect.geoms) == 0:
            continue
        mult_intersect = [m for m in mult_intersect.geoms if
                          not isinstance(m, shpg.Point)]

        # checks that floating point representation is ok
        for m in mult_intersect.copy():
            if isinstance(m, shpg.Polygon):
                # Check area and remove - should be fairly rare
                tmp = gpd.GeoDataFrame(geometry=[m], crs=odf_new.crs)
                area = tmp.to_crs({'proj':'cea'}).area[0]
                assert area < 10
                mult_intersect.remove(m)

        if len(mult_intersect) == 0:
            continue

        # Simplify the geometries if possible
        try:
            mult_intersect = linemerge(mult_intersect)
        except IndexError:
            pass

        # Add each line to the output file
        if isinstance(mult_intersect, shpg.linestring.LineString):
            mult_intersect = shpg.MultiLineString([mult_intersect])
        for line in mult_intersect.geoms:
            assert isinstance(line, shpg.linestring.LineString)
            line = gpd.GeoDataFrame([[major.rgi_id, neighbor.rgi_id, line]],
                                    columns=odf_intersects_cols, crs=odf_new.crs)
            odf_intersects = pd.concat([odf_intersects, line])

odf_intersects = odf_intersects.reset_index(drop=True)

In [ ]:
# assign new ids for intersects - this is arbitrary
baseid = f'RGI2000-v7.0-I-{reg:02d}-'

rp = odf_intersects.representative_point()
coordinates = np.array(list(rp.apply(xy_coord)))
odf_intersects['cenlon'] = coordinates[:, 0]
odf_intersects['cenlat'] = coordinates[:, 1]
    
todo = odf_intersects.index.values
todo_lon = odf_intersects['cenlon'].values
todo_lat = odf_intersects['cenlat'].values
ids = []
while len(todo) > 0:
    dis = haversine(slon, slat, todo_lon, todo_lat)
    idm = np.argmin(dis)
    ids.append(todo[idm])
    slon, slat = todo_lon[idm], todo_lat[idm]
    todo = np.delete(todo, idm)
    todo_lon = np.delete(todo_lon, idm)
    todo_lat = np.delete(todo_lat, idm)

assert len(ids) == len(odf_intersects)
odf_intersects = odf_intersects.loc[ids].copy()

odf_intersects['rgi_id'] = [baseid + f'{l+1:05d}' for l in range(len(odf_intersects))]
odf_intersects = odf_intersects.sort_values(by='rgi_id').reset_index(drop=True)

assert int(odf_intersects.iloc[-1]['rgi_id'].split('-')[-1]) == odf_intersects.iloc[-1].name + 1 

In [ ]:
assert np.alltrue(np.array([g.geom_type for g in odf_intersects.geometry]) == 'LineString')

In [ ]:
gdf = odf_new.set_index('rgi_id')
odf_intersects['utm_zone'] = gdf.loc[odf_intersects['rgi_id_1']]['utm_zone'].values

In [ ]:
# Compute the length
for zone in odf_intersects['utm_zone'].unique():
    sel = odf_intersects.loc[odf_intersects['utm_zone'] == zone]
    odf_intersects.loc[sel.index, 'length_m'] = sel.to_crs({'proj':'utm', 'zone':zone}).length

In [ ]:
# here we could define a minimum length for a intersection line - we decided not to

In [ ]:
odf_intersects = odf_intersects[['rgi_id', 'rgi_id_1', 'rgi_id_2', 'length_m', 'geometry']]
odf_intersects.columns = ['rgi_id', 'rgi_g_id_1', 'rgi_g_id_2', 'length_m', 'geometry']
odf_intersects;

### Create merged shapefile

In [ ]:
# merge outlines
odf_merged = odf_new.dissolve().explode(ignore_index=True).reset_index()

# drop attributes and set others to nan (just to be sure all attributes are recomputed)
attributes_to_keep = ['rgi_id', 'o1region', 'area_km2', 'geometry']
attributes_to_drop = [i for i in odf_merged.columns if i not in attributes_to_keep]
odf_merged.drop(columns=attributes_to_drop, inplace=True)
for attr in attributes_to_keep:
    if attr not in ['o1region', 'geometry']:
        odf_merged[attr] = np.nan

In [ ]:
odf_merged['area'] = odf_merged.to_crs({'proj':'cea'}).area
odf_merged = correct_geoms(odf_merged)

In [ ]:
odf_merged = fix_overaps(odf_merged)

In [ ]:
odf_merged = correct_geoms(odf_merged)

In [ ]:
# define new cenlon and cenlat
def xy_coord(geom):
    x, y = geom.xy
    return x[0], y[0]

rp = odf_merged.representative_point()
coordinates = np.array(list(rp.apply(xy_coord)))
odf_merged['cenlon'] = coordinates[:, 0]
odf_merged['cenlat'] = coordinates[:, 1]

# calculate new area
odf_merged['area_km2'] = odf_merged.to_crs({'proj':'cea'}).area * 1e-6

# Filter
odf_merged = odf_merged.loc[odf_merged['area_km2'] >= 0.01].copy()

# check that total area is unchanged
assert np.allclose(odf_new.area_km2.sum(),
                   odf_merged.area_km2.sum())

In [ ]:
# assign new ids for merged glacier complexes - this is arbitrary
baseid = f'RGI2000-v7.0-C-{reg:02d}-'
count = 1

rp = odf_merged.representative_point()
rp = rp.to_frame('geometry')
rp['orig_index'] = odf_merged.index

total = 0

for i, s in sreg.iterrows():
    
    intersect = gpd.overlay(rp, sreg.loc[[i]], how='intersection')
    odf_sreg = odf_merged.loc[intersect['orig_index']].copy()
    odf_sreg['o2region'] = s['o2region']
    
    total += len(odf_sreg)
    
    if len(odf_sreg) == 0:
        # 19-05 Ross Ice Shelf one of them
        continue
    
    # Ids generation
    # Left most point and we start from there
    slon, slat = odf_sreg.loc[odf_sreg.cenlon == odf_sreg.cenlon.min()][['cenlon', 'cenlat']].iloc[0].values
    
    todo = odf_sreg.index.values
    todo_lon = odf_sreg['cenlon'].values
    todo_lat = odf_sreg['cenlat'].values
    ids = []
    while len(todo) > 0:
        dis = haversine(slon, slat, todo_lon, todo_lat)
        idm = np.argmin(dis)
        ids.append(todo[idm])
        slon, slat = todo_lon[idm], todo_lat[idm]
        todo = np.delete(todo, idm)
        todo_lon = np.delete(todo_lon, idm)
        todo_lat = np.delete(todo_lat, idm)
    
    assert len(ids) == len(odf_sreg)
    odf_sreg = odf_sreg.loc[ids].copy()
    
    odf_sreg['rgi_id'] = [baseid + f'{l+count:05d}' for l in range(len(odf_sreg))]
    count += len(odf_sreg)
    odf_merged.loc[odf_sreg.index, 'rgi_id'] = odf_sreg['rgi_id']
    odf_merged.loc[odf_sreg.index, 'o2region'] = odf_sreg['o2region']
    
odf_merged = odf_merged.sort_values(by='rgi_id').reset_index(drop=True)

In [ ]:
if odf_merged.rgi_id.isna().sum() > 0:
    # Some merged glaciers are not within a subregion
    # Happens eg in rgi14 where the regions are crap
    # pick the nearest glacier and apply the same region
    for i, s in odf_merged.loc[odf_merged.rgi_id.isna()].copy().iterrows():
        dis = haversine(s.cenlon, s.cenlat, odf_merged.cenlon, odf_merged.cenlat)
        assert dis.sort_values().iloc[0] == 0
        assert dis.sort_values().iloc[1] > 0
        for ii, d in zip(dis.sort_values().iloc[1:].index, dis.sort_values().iloc[1:]):
            totry = odf_merged.loc[ii]
            if totry.o2region != '':
                odf_merged.loc[i, 'rgi_id'] = baseid + f'{count:05d}'
                odf_merged.loc[i, 'o2region'] = totry.o2region
                count += 1
                break
assert odf_merged.rgi_id.isna().sum() == 0
assert int(odf_merged.rgi_id.iloc[-1].split('-')[-1]) == odf_merged.index[-1]+1

In [ ]:
not_valid = ~ odf_merged.is_valid
assert not_valid.sum() == 0, 'Merged product wrong geoms'

In [ ]:
# UTM zone
utms = []
for lat, lon in zip(odf_merged.cenlat, odf_merged.cenlon):
    _, _, nz, _ = utm.from_latlon(lat, lon)
    utms.append(nz)
odf_merged['utm_zone'] = utms

In [ ]:
assert np.alltrue(np.array([g.geom_type for g in odf_merged.geometry]) == 'Polygon')

In [ ]:
odf_merged = odf_merged[['rgi_id', 'o1region', 'o2region', 'cenlon', 'cenlat', 'utm_zone', 'area_km2', 'geometry']].reset_index(drop=True)
odf_merged.iloc[[0]].T;

### Create conversion list between individual glacier ids and glacier complexes

In [ ]:
# Compute RGI7 - RGI6 overlaps
overlaps_merged = overlaps_helpers.compute_cross_overlaps(odf_merged.geometry, odf_new.geometry)

In [ ]:
assert len(overlaps_merged['i'].unique()) == len(odf_merged)
assert overlaps_merged.j_area_fraction.min() > 0.999
assert len(overlaps_merged) == len(odf_new)

In [ ]:
overlaps_merged['i'] = odf_merged['rgi_id'].iloc[overlaps_merged['i']].values
overlaps_merged['j'] = odf_new['rgi_id'].iloc[overlaps_merged['j']].values

In [ ]:
individual_ids_per_complex_dict = {}
for cid in overlaps_merged['i'].sort_values().unique():
    individual_ids_per_complex_dict[cid] = overlaps_merged.loc[overlaps_merged['i'] == cid]['j'].values.tolist()

In [ ]:
# check that every individual glacier was assigned to one and only one complex
assigned_ids = [i for sublist in list(individual_ids_per_complex_dict.values())
                for i in sublist]
assert len(assigned_ids) == len(odf_new.rgi_id)
assert len(np.unique(assigned_ids)) == len(odf_new.rgi_id)

## Write out and tar 

In [ ]:
reg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions')
reg_file = gpd.read_file(reg_file)
reg_file = reg_file.loc[reg_file.o1region == f'{reg:02d}'].iloc[0]

### Glacier product 

In [ ]:
dd = mkdir(f'{output_dir}/RGI2000-v7.0-G-{reg_file.long_code}/', reset=True)

print('Writing...')
odf_new.to_file(dd + f'RGI2000-v7.0-G-{reg_file.long_code}.shp')
odf_subm.to_csv(dd + f'RGI2000-v7.0-G-{reg_file.long_code}-submission_info.csv', quoting=csv.QUOTE_NONNUMERIC)
odf_links.to_csv(dd + f'RGI2000-v7.0-G-{reg_file.long_code}-rgi6_links.csv', quoting=csv.QUOTE_NONNUMERIC)
odf_new.drop('geometry', axis=1).set_index('rgi_id').to_csv(dd + f'RGI2000-v7.0-G-{reg_file.long_code}-attributes.csv', quoting=csv.QUOTE_NONNUMERIC)
shutil.copyfile('../README_tpl.md', dd + f'README.md')
shutil.copyfile('../rgi7_attributes_metadata.json', dd + f'RGI2000-v7.0-G-{reg_file.long_code}-attributes_metadata.json')
shutil.copyfile('../rgi7_submission_info_metadata.json', dd + f'RGI2000-v7.0-G-{reg_file.long_code}-submission_info_metadata.json')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI2000-v7.0-G-{reg_file.long_code}.tar.gz', '-C', output_dir, f'RGI2000-v7.0-G-{reg_file.long_code}']))

In [ ]:
odf_new;

In [ ]:
import glob, os
for f in glob.glob( f"{output_dir_tar}/*.properties"):
    os.remove(f)

## Save intersects

In [ ]:
dd = mkdir(f'{output_dir}/RGI2000-v7.0-I-{reg_file.long_code}/', reset=True)

print('Writing...')

# save intersects
odf_intersects.to_file(dd + f'RGI2000-v7.0-I-{reg_file.long_code}.shp')
odf_intersects.drop('geometry', axis=1).set_index('rgi_id').to_csv(dd + f'RGI2000-v7.0-I-{reg_file.long_code}-attributes.csv', quoting=csv.QUOTE_NONNUMERIC)
shutil.copyfile('../README_tpl.md', dd + f'README.md')
shutil.copyfile('../rgi7_intersects_attributes_metadata.json', dd + f'RGI2000-v7.0-I-{reg_file.long_code}-attributes_metadata.json')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI2000-v7.0-I-{reg_file.long_code}.tar.gz', '-C', output_dir, f'RGI2000-v7.0-I-{reg_file.long_code}']))

## Save glacier complex

In [ ]:
dd = mkdir(f'{output_dir}/RGI2000-v7.0-C-{reg_file.long_code}/', reset=True)

print('Writing...')

# save merged product with attribute table
odf_merged.to_file(dd + f'RGI2000-v7.0-C-{reg_file.long_code}.shp')
odf_merged.drop('geometry', axis=1).set_index('rgi_id').to_csv(dd + f'RGI2000-v7.0-C-{reg_file.long_code}-attributes.csv', quoting=csv.QUOTE_NONNUMERIC)
shutil.copyfile('../README_tpl.md', dd + f'README.md')

# save conversion list between G and C
fp = dd + f'RGI2000-v7.0-C-{reg_file.long_code}-CtoG_links.json'
with open(fp, 'w') as f:
    json.dump(individual_ids_per_complex_dict, f, indent=2)
shutil.copyfile('../rgi7_complex_attributes_metadata.json', dd + f'RGI2000-v7.0-C-{reg_file.long_code}-attributes_metadata.json')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI2000-v7.0-C-{reg_file.long_code}.tar.gz', '-C', output_dir, f'RGI2000-v7.0-C-{reg_file.long_code}']))